# THỰC HÀNH BUỔI 4

## 1.Thực hành code

In [3]:
# các thư viện cần
import nltk
nltk.download('punkt')
nltk.download('punk_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Error loading punk_tab: Package 'punk_tab' not found in
[nltk_data]     index


False

In [5]:
from nltk.util import ngrams
from collections import Counter
import math
import re

In [ ]:
import re
import nltk

def preprocess_text(text):
    # Chuyển đổi tất cả các ký tự trong chuỗi thành chữ thường
    text = text.lower()
    
    # Loại bỏ tất cả dấu câu, chỉ giữ lại chữ cái, số và khoảng trắng
    text = re.sub(r'[^\w\s]', '', text)
    """\w: Các ký tự chữ cái và số.s
    	\s: Khoảng trắng.
        [^\w\s]: Bất kỳ ký tự nào không phải chữ, số hoặc khoảng trắng sẽ bị loại bỏ (ví dụ: dấu chấm câu)."""
    # Tokenize (tách văn bản thành các từ riêng lẻ)
    tokens = nltk.word_tokenize(text)
    
    # Thêm token đặc biệt vào đầu và cuối câu
    tokens = ['<s>'] + tokens + ['</s>']
    
    return tokens


In [9]:
from collections import Counter
from nltk.util import ngrams

def build_ngram_model(tokens, n):
    """
    Xây dựng mô hình n-gram từ danh sách tokens.
    
    Parameters:
        tokens (list): Danh sách các từ đã tiền xử lý.
        n (int): Bậc của n-gram (unigram, bigram, trigram,...).
    
    Returns:
        Counter: Một dictionary chứa n-grams và số lần xuất hiện của chúng.
    """
    n_grams = list(ngrams(tokens, n))  # Tạo danh sách các n-gram từ danh sách tokens
    model = Counter(n_grams)  # Đếm tần suất xuất hiện của từng n-gram
    return model  # Trả về mô hình n-gram


def calculate_ngram_probability(model, n_minus_1_model, vocab_size, ngram, k=1):
    """
    Tính xác suất có điều kiện của một n-gram dựa trên mô hình n-gram và Laplace smoothing.
    
    Parameters:
        model (Counter): Mô hình n-gram ở bậc n (ví dụ: bigram, trigram).
        n_minus_1_model (Counter): Mô hình n-gram ở bậc (n-1) (ví dụ: unigram nếu đang xét bigram).
        vocab_size (int): Kích thước từ vựng của tập dữ liệu.
        ngram (tuple): Một n-gram cần tính xác suất.
        k (int, optional): Hệ số điều chỉnh Laplace smoothing (default = 1).
    
    Returns:
        float: Xác suất có điều kiện của n-gram.
    """
    count_ngram = model.get(ngram, 0) + k  # Số lần xuất hiện của n-gram (thêm k để tránh xác suất bằng 0)
    
    prefix = ngram[:-1]  # Phần tiền tố của n-gram (ví dụ: với trigram ('a', 'b', 'c') -> prefix ('a', 'b'))
    count_prefix = n_minus_1_model.get(prefix, 0) + (vocab_size * k)  # Số lần xuất hiện của tiền tố (áp dụng Laplace)

    return count_ngram / count_prefix  # Xác suất có điều kiện của n-gram


In [ ]:
def predict_next_word(model, n_minus_1_model, vocab, prefix, n, k=1):
    # Kiểm tra xem tiền tố (prefix) có tồn tại trong mô hình (n-1)-gram không
    if prefix not in n_minus_1_model:
        return None  # Nếu không có, không thể dự đoán -> trả về None
    
    candidates = {}  # Tạo dictionary để lưu xác suất của các từ tiếp theo
    
    # Duyệt qua từng từ trong từ vựng
    for word in vocab:
        # Tính xác suất có điều kiện của n-gram dựa trên tiền tố và từ hiện tại
        prob = calculate_ngram_probability(model, n_minus_1_model, len(vocab), prefix + (word,), k)
        candidates[word] = prob  # Lưu xác suất của từ vào dictionary
    
    # Trả về từ có xác suất cao nhất (nếu có từ hợp lệ)
    return max(candidates, key=candidates.get) if candidates else None

"""🔹 Cách hoạt động
Kiểm tra xem tiền tố (prefix) có trong mô hình (n-1)-gram không

Nếu không có, trả về None (vì không có dữ liệu để dự đoán).

Duyệt qua tất cả các từ trong vocab

Tính xác suất của từng từ dựa trên calculate_gram_probability.

Lưu xác suất vào candidates.

Chọn từ có xác suất cao nhất

Trả về từ có giá trị lớn nhất trong candidates (nếu danh sách không rỗng)."""

In [11]:
def calculate_perplexity(model, n_minus_1_model, test_tokens, n, vocab_size, k=1):
    # Kiểm tra nếu danh sách test_tokens rỗng, trả về vô cực (inf) để tránh lỗi tính toán
    if not test_tokens:
        return float('inf')  
    
    # Tạo danh sách các n-grams từ tập dữ liệu kiểm tra
    test_ngrams = list(ngrams(test_tokens, n))
    
    # Nếu không có n-gram nào được tạo ra, trả về vô cực
    if not test_ngrams:
        return float('inf') 
    
    log_prob = 0  # Khởi tạo tổng log xác suất
    
    # Duyệt qua từng n-gram trong tập kiểm tra
    for ngram in test_ngrams:
        # Tính xác suất có điều kiện của n-gram
        prob = calculate_ngram_probability(model, n_minus_1_model, vocab_size, ngram, k)
        
        # Nếu xác suất > 0, lấy log bình thường
        if prob > 0:
            log_prob += math.log(prob)
        else:
            # Tránh log(0) bằng cách thay thế bằng giá trị rất nhỏ (1e-10)
            log_prob += math.log(1e-10)  

    # Tính perplexity
    return math.exp(-log_prob / len(test_ngrams))


In [14]:
if __name__ == "__main__":
    # Tải xuống bộ tokenizer 'punkt' từ NLTK nếu chưa có
    nltk.download('punkt')
    
    # Khởi tạo văn bản đầu vào
    text = "hôm nay trời đẹp. hôm nay tôi đi thực hành NLP ở phòng máy 10. Thực hành NLP rất thú vị"
    
    # Tiền xử lý văn bản: chuyển thành chữ thường, loại bỏ dấu câu, tokenization
    tokens = preprocess_text(text)

    # Xây dựng mô hình bigram (n-gram với n=2)
    bigram_model = build_ngram_model(tokens, 2)

    # Xây dựng mô hình unigram (n-gram với n=1)
    unigram_model = build_ngram_model(tokens, 1)

    # Tạo tập từ vựng từ danh sách token
    vocab = set(tokens)

    # In mô hình bigram đã tạo
    print("Bigram model:", bigram_model)

    # Dự đoán từ tiếp theo sau từ 'tôi' bằng mô hình bigram
    next_word = predict_next_word(bigram_model, unigram_model, vocab, ('tôi',), 2)
    print("Từ dự đoán tiếp theo sau 'tôi':", next_word)

    # Văn bản thử nghiệm để tính perplexity
    test_text = "tôi thích AI"

    # Tiền xử lý văn bản thử nghiệm
    test_tokens = preprocess_text(test_text)

    # Tính perplexity của mô hình trên văn bản thử nghiệm
    perplexity = calculate_perplexity(bigram_model, unigram_model, test_tokens, 2, len(vocab))
    print("Perplexity của mô hình:", perplexity)


Bigram model: Counter({('hôm', 'nay'): 2, ('thực', 'hành'): 2, ('hành', 'nlp'): 2, ('<s>', 'hôm'): 1, ('nay', 'trời'): 1, ('trời', 'đẹp'): 1, ('đẹp', 'hôm'): 1, ('nay', 'tôi'): 1, ('tôi', 'đi'): 1, ('đi', 'thực'): 1, ('nlp', 'ở'): 1, ('ở', 'phòng'): 1, ('phòng', 'máy'): 1, ('máy', '10'): 1, ('10', 'thực'): 1, ('nlp', 'rất'): 1, ('rất', 'thú'): 1, ('thú', 'vị'): 1, ('vị', '</s>'): 1})
Từ dự đoán tiếp theo sau 'tôi': đi
Perplexity của mô hình: 18.493242008906932


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# ĐỈNH CAO CÔNG NGHỆ

In [15]:
import nltk
import re
import math
from collections import Counter
from nltk.util import ngrams

nltk.download('punkt')

def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    tokens = nltk.word_tokenize(text)
    return ['<s>'] + tokens + ['</s>']

def build_ngram_model(tokens, n):
    n_grams = list(ngrams(tokens, n))
    return Counter(n_grams)

def calculate_ngram_probability(model, n_minus_1_model, vocab_size, ngram, k=1):
    count_ngram = model.get(ngram, 0) + k
    count_prefix = sum(n_minus_1_model.values()) + (vocab_size * k)
    return count_ngram / count_prefix

def predict_next_word(model, n_minus_1_model, vocab, prefix, k=1):
    candidates = {}
    for word in vocab:
        prob = calculate_ngram_probability(model, n_minus_1_model, len(vocab), prefix + (word,), k)
        candidates[word] = prob
    return sorted(candidates.items(), key=lambda x: x[1], reverse=True)  # Sắp xếp theo xác suất giảm dần

if __name__ == "__main__":
    text = "hôm nay trời đẹp. hôm nay tôi đi thực hành NLP ở phòng máy 10. Thực hành NLP rất thú vị"
    tokens = preprocess_text(text)

    bigram_model = build_ngram_model(tokens, 2)
    unigram_model = build_ngram_model(tokens, 1)
    vocab = set(tokens)

    user_input = input("Nhập một từ: ").lower()
    suggestions = predict_next_word(bigram_model, unigram_model, vocab, (user_input,))
    
    print("Các từ gợi ý tiếp theo kèm xác suất:")
    for word, prob in suggestions[:5]:  # Hiển thị top 5 từ có xác suất cao nhất
        print(f"{word}: {prob:.4f}")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Các từ gợi ý tiếp theo kèm xác suất:
nay: 0.0244
đi: 0.0244
<s>: 0.0244
hành: 0.0244
rất: 0.0244


In [1]:
import tkinter as tk
from collections import Counter
import nltk
from nltk.util import ngrams

nltk.download('punkt')

def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens

def build_ngram_model(tokens, n):
    n_grams = list(ngrams(tokens, n))
    model = Counter(n_grams)
    return model

def predict_next_word(model, prefix):
    candidates = {k[-1]: v for k, v in model.items() if k[:-1] == prefix}
    return sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:5]

def update_suggestions(event=None):
    prefix = entry_var.get().strip().split()
    if not prefix:
        suggestion_list.delete(0, tk.END)
        return

    prefix = tuple(prefix[-1:])
    suggestions = predict_next_word(bigram_model, prefix)

    suggestion_list.delete(0, tk.END)
    for word, _ in suggestions:
        suggestion_list.insert(tk.END, word)

    if suggestion_list.size() > 0:
        suggestion_list.selection_set(0)
        suggestion_list.activate(0)

def on_tab(event):
    if suggestion_list.size() > 0:
        try:
            index = suggestion_list.curselection()[0]
            selected_word = suggestion_list.get(index)
        except IndexError:
            selected_word = suggestion_list.get(0)

        current_text = entry_var.get().strip()
        new_text = f"{current_text} {selected_word}"
        entry_var.set(new_text)
        entry.icursor(tk.END)
        suggestion_list.delete(0, tk.END)
        return "break"

def on_arrow_key(event):
    if suggestion_list.size() == 0:
        return

    index = suggestion_list.curselection()
    if not index:
        current_index = 0
    else:
        current_index = index[0]

    if event.keysym == "Down":
        new_index = min(current_index + 1, suggestion_list.size() - 1)
    elif event.keysym == "Up":
        new_index = max(current_index - 1, 0)
    else:
        return

    suggestion_list.selection_clear(0, tk.END)
    suggestion_list.selection_set(new_index)
    suggestion_list.activate(new_index)
    return "break"  # Ngăn mặc định di chuyển con trỏ

# Đọc dữ liệu từ file
with open("data_ctut.txt", "r", encoding="utf-8") as file:
    text = file.read()

tokens = preprocess_text(text)
bigram_model = build_ngram_model(tokens, 2)

# Giao diện Tkinter
root = tk.Tk()
root.title("Gợi ý từ")

entry_var = tk.StringVar()
entry = tk.Entry(root, textvariable=entry_var, font=("Arial", 14), width=50)
entry.pack(padx=10, pady=5)

suggestion_list = tk.Listbox(root, font=("Arial", 14), width=50)
suggestion_list.pack(padx=10, pady=5)

# Gắn sự kiện
entry.bind("<KeyRelease>", update_suggestions)
entry.bind("<Tab>", on_tab)
entry.bind("<Up>", on_arrow_key)
entry.bind("<Down>", on_arrow_key)

root.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
import tkinter as tk
from tkinter import ttk
from collections import Counter
import nltk
from nltk.util import ngrams

nltk.download('punkt')

# ========== Xử lý ngôn ngữ ==========
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens

def build_ngram_model(tokens, n):
    n_grams = list(ngrams(tokens, n))
    return Counter(n_grams)

def predict_next_word(model, prefix):
    prefix = tuple(prefix)
    candidates = {k[-1]: v for k, v in model.items() if k[:-1] == prefix}
    return sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:5]

# ========== Hàm sự kiện ==========
def update_suggestions(event=None):
    words = entry_var.get().strip().split()
    if not words or len(words) < n - 1:
        suggestion_list.delete(0, tk.END)
        return

    prefix = words[-(n - 1):]
    suggestions = predict_next_word(ngram_model, prefix)

    suggestion_list.delete(0, tk.END)
    for word, _ in suggestions:
        suggestion_list.insert(tk.END, word)

    if suggestion_list.size() > 0:
        suggestion_list.selection_set(0)
        suggestion_list.activate(0)

def on_tab(event):
    if suggestion_list.size() == 0:
        return

    try:
        index = suggestion_list.curselection()[0]
    except IndexError:
        index = 0

    selected_word = suggestion_list.get(index)
    current_text = entry_var.get().strip()
    new_text = f"{current_text} {selected_word}"
    entry_var.set(new_text)
    entry.icursor(tk.END)
    suggestion_list.delete(0, tk.END)
    return "break"

def on_arrow_key(event):
    if suggestion_list.size() == 0:
        return

    index = suggestion_list.curselection()
    current_index = index[0] if index else 0

    if event.keysym == "Down":
        new_index = min(current_index + 1, suggestion_list.size() - 1)
    elif event.keysym == "Up":
        new_index = max(current_index - 1, 0)
    else:
        return

    suggestion_list.selection_clear(0, tk.END)
    suggestion_list.selection_set(new_index)
    suggestion_list.activate(new_index)
    return "break"

def on_click_suggestion(event):
    try:
        index = suggestion_list.curselection()[0]
        selected_word = suggestion_list.get(index)
        current_text = entry_var.get().strip()
        new_text = f"{current_text} {selected_word}"
        entry_var.set(new_text)
        entry.icursor(tk.END)
        suggestion_list.delete(0, tk.END)
    except IndexError:
        pass

# ========== Đọc dữ liệu và tạo mô hình ==========
with open("data_ctut.txt", "r", encoding="utf-8") as file:
    text = file.read()

tokens = preprocess_text(text)
n = 2  # Đổi thành 3 nếu muốn dùng trigram
ngram_model = build_ngram_model(tokens, n)

# ========== Giao diện Tkinter ==========
root = tk.Tk()
root.title("Gợi ý từ thông minh")
root.geometry("600x300")
root.resizable(False, False)

# Entry
entry_var = tk.StringVar()
entry = tk.Entry(root, textvariable=entry_var, font=("Arial", 14), width=50)
entry.pack(pady=10)

# Scrollbar + Listbox
frame = tk.Frame(root)
frame.pack()

scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL)
suggestion_list = tk.Listbox(frame, font=("Arial", 14), width=50, height=6, yscrollcommand=scrollbar.set)
scrollbar.config(command=suggestion_list.yview)

suggestion_list.pack(side=tk.LEFT, fill=tk.BOTH)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# ========== Gắn sự kiện ==========
entry.bind("<KeyRelease>", update_suggestions)
entry.bind("<Tab>", on_tab)
entry.bind("<Up>", on_arrow_key)
entry.bind("<Down>", on_arrow_key)
suggestion_list.bind("<Double-Button-1>", on_click_suggestion)

# Chạy giao diện
root.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [8]:
import tkinter as tk
from collections import Counter
import nltk
from nltk.util import ngrams

nltk.download('punkt')

# ======== NLP xử lý =========
def preprocess_text(text):
    tokens = nltk.word_tokenize(text.lower())
    return tokens

def build_ngram_model(tokens, n):
    n_grams = list(ngrams(tokens, n))
    return Counter(n_grams)

def predict_next_word(model, prefix):
    prefix = tuple(prefix)
    candidates = {k[-1]: v for k, v in model.items() if k[:-1] == prefix}
    return sorted(candidates.items(), key=lambda x: x[1], reverse=True)[:5]

# ======== Sự kiện =========
def update_suggestions(event=None):
    words = entry_var.get().strip().split()
    if not words or len(words) < n - 1:
        suggestion_list.delete(0, tk.END)
        return

    prefix = words[-(n - 1):]
    suggestions = predict_next_word(ngram_model, prefix)

    suggestion_list.delete(0, tk.END)
    for word, _ in suggestions:
        suggestion_list.insert(tk.END, word)

    # Mặc định chọn từ đầu
    if suggestion_list.size() > 0:
        suggestion_list.selection_set(0)
        suggestion_list.activate(0)

# Tab → chèn từ đang chọn
def on_tab(event):
    if suggestion_list.size() == 0:
        return "break"

    try:
        index = suggestion_list.curselection()[0]
        selected_word = suggestion_list.get(index)
    except IndexError:
        return "break"

    current_text = entry_var.get().strip()
    new_text = f"{current_text} {selected_word}"
    entry_var.set(new_text)
    entry.icursor(tk.END)
    suggestion_list.delete(0, tk.END)
    return "break"

# ↑ ↓ → di chuyển đúng 1 dòng
def on_arrow_key(event):
    if suggestion_list.size() == 0:
        return "break"

    try:
        index = suggestion_list.curselection()[0]
    except IndexError:
        index = -1

    if event.keysym == "Down" and index < suggestion_list.size() - 1:
        index += 1
    elif event.keysym == "Up" and index > 0:
        index -= 1
    else:
        return "break"

    suggestion_list.selection_clear(0, tk.END)
    suggestion_list.selection_set(index)
    suggestion_list.activate(index)
    suggestion_list.see(index)
    return "break"

# Nhấn chuột → chọn từ
def on_click_suggestion(event):
    try:
        index = suggestion_list.curselection()[0]
        selected_word = suggestion_list.get(index)
        current_text = entry_var.get().strip()
        new_text = f"{current_text} {selected_word}"
        entry_var.set(new_text)
        entry.icursor(tk.END)
        suggestion_list.delete(0, tk.END)
    except IndexError:
        pass

# ======== Dữ liệu & mô hình =========
with open("data_ctut.txt", "r", encoding="utf-8") as file:
    text = file.read()

tokens = preprocess_text(text)
n = 2
ngram_model = build_ngram_model(tokens, n)

# ======== Giao diện =========
root = tk.Tk()
root.title("Gợi ý từ thông minh")
root.geometry("600x300")

entry_var = tk.StringVar()
entry = tk.Entry(root, textvariable=entry_var, font=("Arial", 14), width=50)
entry.pack(pady=10)

frame = tk.Frame(root)
frame.pack()

scrollbar = tk.Scrollbar(frame, orient=tk.VERTICAL)
suggestion_list = tk.Listbox(frame, font=("Arial", 14), width=50, height=6, yscrollcommand=scrollbar.set)
scrollbar.config(command=suggestion_list.yview)
suggestion_list.pack(side=tk.LEFT, fill=tk.BOTH)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

# ======== Gắn sự kiện =========
entry.bind("<KeyRelease>", update_suggestions)
entry.bind("<Tab>", on_tab)
entry.bind("<Up>", on_arrow_key)
entry.bind("<Down>", on_arrow_key)
suggestion_list.bind("<Double-Button-1>", on_click_suggestion)

# Chạy ứng dụng
root.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
